In [2]:
# Install the required packages
%pip install pandas Office365-REST-Python-Client
%pip install pyyaml


     ---------------------------------------- 1.2/1.2 MB 9.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import the required libraries
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
import pandas as pd


In [6]:
import yaml

config_file = '../config.yaml'

# Load configuration file
with open(config_file, 'r') as config_file:
    config = yaml.safe_load(config_file)

# Set the site URL
site_url = config['site_url']

# Set the username and password
username = config['username']
password = config['password']

# Set the Excel file path
input_file_path = config['file_path']


Site URL: https://your-sharepoint-site.sharepoint.com/sites/your-site/
Username: your_username@yourdomain.com
Password: your_password
File Path: /sites/your-site/Shared Documents/YourFolder/your_excel_file.xlsx


In [ ]:
# Authentication context
auth_context = AuthenticationContext(site_url)
auth_context.acquire_token_for_user(username, password)

# Create client context
client_context = ClientContext(site_url, auth_context)


In [ ]:
# Load Excel data into DataFrame
excel_file = client_context.web.get_file_by_server_relative_url(input_file_path)
response = excel_file.download()
data = response.content
df = pd.read_excel(data)
